# Elevation Interpolation and Accuracy Assessment

###### Help provided by ChatGPT

## Import Statments

In [ ]:
import os
import psycopg2
from psycopg2 import sql
import arcpy
import psycopg2
import random

## Path

In [1]:
#location to current directory (relative path)
file_path = os.path.dirname(arcpy.mp.ArcGISProject('CURRENT').filePath)
os.chdir(file_path)
#absolute path for geodatabase
arcpy.env.workspace = file_path
arcpy.management.CreateFileGDB(file_path, "Lab3.gdb")

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\ArcIILab2\\Lab3.gdb'>

## Database Connection

In [2]:
#step1: Connection to database
database_key = 'loganandgreg'
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password=database_key,
    host="35.232.21.197",
    port="5432"
)

## Pulling Data from Database

In [3]:
#saving created datasets
output_feature_class = "TempLab3"
output_database = "Lab3.gdb"
#create an empty feature class (stores retrieved data)
arcpy.CreateFeatureclass_management(output_database, output_feature_class, "POINT", spatial_reference=arcpy.SpatialReference(4326))
#add fields to store WKT geometry (latitude/longitude of points) and grid code (value of cell)
arcpy.AddField_management(output_feature_class, "WKT_GEOM", "TEXT")
arcpy.AddField_management(output_feature_class, "GRID_CODE", "FLOAT")
#cursor is interaction to database (allows us to make SQL statements)
cur = conn.cursor()
#SQL query to retrieve data from the table as temperature and longitude and latitude as WKT (in that order)
#pulling data for one data (maximum temperature data for one date)
select_query = """SELECT value_tmax, CONCAT('POINT(', longitude, ' ', latitude, ')') AS WKT
FROM temp_data
WHERE date = '2023-12-20T00:00:00';"""
#execute SQL query
cur.execute(select_query)
#inserting using arcpy.da.Insert: SHAPE@WKT is the actual point, WKT_GEOM is storing coordinates, GRID_Code value of raster cell-- stores in output (TempLab3)
with arcpy.da.InsertCursor(output_feature_class, ["SHAPE@WKT", "WKT_GEOM", "GRID_CODE"]) as cursor:
    #fetch all rows in table returned from SQL
    rows = cur.fetchall()
    #print(len(rows))-- this was to test that I had the right about of rows
    #retrieved data
    for row in rows:
        wkt_geom = row[1] #this is wkt_geom (first index)
        grid_code = row[0] #this is grid_code (second index)
        #insert the row into the output feature class (TempLab3)
        cursor.insertRow([wkt_geom, wkt_geom, grid_code])
#close the cursor/database connection
cur.close()
conn.close()

175


## Sampling Data

#### Justification: 90/10 Split
##### By allocating 90% of the data for training, the model has enough data to learn from the underlying patterns and relationships. Additionally, a larger training model can reduce bias in the model. More data means better models. The 10% sample is good for examining generalizations within datasets, as it learns relevant patterns without overfitting to the training data. This training-testing split ratio overall is good for model evaluation and model validation, as elevation has a lot of variance and a larger dataset can identify and interpret anomalies and classify the remaining data correctly.

In [4]:
#input feature class and output feature classes
input_feature_class = "TempLab3"
#10% - Testing Dataset
output_sample_fc = "RandomSampledFeatures2"
#90% - Training data
output_remaining_fc = "RemainingFeatures2"
#gets the number of features in the input class starting at index 0 as an integer (counts rows)
total_count = int(arcpy.GetCount_management(input_feature_class)[0])
#create empty list to store each row we want to subset (stores all data)
all_features = []
#creates search cursor to iterate through the input feature class (store as list and shuffle data at random) (copy of dataset)
with arcpy.da.SearchCursor(input_feature_class, ["SHAPE@WKT", "GRID_CODE"]) as cursor:
    for row in cursor:
        #extract WKT geometry and temperature (grid_code)
        wkt_geometry = row[0]
        Temp = row[1]
        #append WKT geometry and elevation to the all features list
        all_features.append((wkt_geometry, Temp))
#randomize data
random.shuffle(all_features)
#calculate sample size (10%)
sample_size = int(total_count * 0.1)
#split the shuffled list into sampled features and remaining features
sampled_features = all_features[:sample_size]
remaining_features = all_features[sample_size:]
#create a new feature class to store the random sample as a point feature class
arcpy.CreateFeatureclass_management(arcpy.env.workspace, output_sample_fc, "POINT", spatial_reference=arcpy.Describe(input_feature_class).spatialReference)
#create new field titled elevation
arcpy.AddField_management(output_sample_fc, "Temp", "DOUBLE")
#insert the randomly sampled features into the output sample feature class
with arcpy.da.InsertCursor(output_sample_fc, ["SHAPE@WKT", "Temp"]) as cursor:
    for wkt_geometry, Temp in sampled_features:
        cursor.insertRow((wkt_geometry, Temp))
print("Randomly sampled features have been saved to", output_sample_fc)
#create a new feature class to store the remaining features
arcpy.CreateFeatureclass_management(arcpy.env.workspace, output_remaining_fc, "POINT", spatial_reference=arcpy.Describe(input_feature_class).spatialReference)
#add fields to the output remaining feature class
arcpy.AddField_management(output_remaining_fc, "Temp", "DOUBLE")
#insert the remaining features into the output remaining feature class
with arcpy.da.InsertCursor(output_remaining_fc, ["SHAPE@WKT", "Temp"]) as cursor:
    for wkt_geometry, Temp in remaining_features:
        cursor.insertRow((wkt_geometry, Temp))
print("Remaining features have been saved to", output_remaining_fc)

Randomly sampled features have been saved to RandomSampledFeatures2
Remaining features have been saved to RemainingFeatures2


## Leave One Out Cross Validation

In [5]:
#use training data to perform analysis
#"Generates various interpolation results from input point features and a field. The interpolation results are then compared and ranked using customizable criteria based on cross validation statistics (ESRI, 2024)."
arcpy.ga.ExploratoryInterpolation(
    in_features="RemainingFeatures2",
    value_field="Temp",
    out_cv_table= (arcpy.env.workspace + "\\Lab3.gdb\\exploratoryinterpolation2"),
    out_geostat_layer=None,
    interp_methods="KERNEL_INTERPOLATION;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\ArcIILab2\\Lab3.gdb\\exploratoryinterpolation2'>

## Interpolation

Definitions: 

Kriging: "An interpolation technique in which the surrounding measured values are weighted to derive a predicted value for an unmeasured location. Weights are based on the distance between the measured points, the prediction locations, and the overall spatial arrangement among the measured points (ESRI, 2024)."

KernelInterpolationWtihBarriers: "A moving window predictor that uses the shortest distance between points so that points on either side of the line barriers are connected.Kernel Interpolation is a variant of a first-order Local Polynomial Interpolation in which instability in the calculations is prevented using a method similar to the one used in the ridge regression to estimate the regression coefficients (ESRI, 2024)."

Inverse distance weighted: "(IDW) interpolation determines cell values using a linearly weighted combination of a set of sample points. The weight is a function of inverse distance. The surface being interpolated should be that of a locationally dependent variable (ESRI, 2024)."

In [11]:
#kriging
Kriging_Temp = arcpy.sa.Kriging(
    in_point_features="RemainingFeatures2",
    z_field="Temp",
    kriging_model="LinearDrift 0.021788 # # #",
    cell_size=0.1,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)

#kernel
arcpy.ga.KernelInterpolationWithBarriers(
    in_features="RemainingFeatures2",
    z_field="Temp",
    out_ga_layer=None,
    out_raster=arcpy.env.workspace + "\\ArcIILab2.gdb\\Kernel_Temp",
    cell_size=0.1,
    in_barrier_features=None,
    kernel_function="POLYNOMIAL5",
    bandwidth=None,
    power=1,
    ridge=50,
    output_type="PREDICTION"
)

#IDW
arcpy.ga.IDW(
    in_features="RemainingFeatures2",
    z_field="Temp",
    out_ga_layer=None,
    out_raster=arcpy.env.workspace + "\\ArcIILab2.gdb\\IDW_Temp",
    cell_size=0.1,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=2.19318511074646 S_MINOR=2.19318511074646 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

## Database Connection

In [12]:
#create connection to cloud database (On google cloud-- SQL database)
out_folder_path = arcpy.env.workspace
out_name = "Lab3.sde"
database_platform = "POSTGRESQL"
instance = "35.232.21.197"
account_authentication = "DATABASE_AUTH"
username = "postgres"
password = "loganandgreg"
save_user_pass = "SAVE_USERNAME"
database = "gis5572"
#actual database connection
arcpy.management.CreateDatabaseConnection(out_folder_path, out_name, database_platform, instance, account_authentication, username, password, save_user_pass, database)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\ArcIILab2\\Lab3.sde'>

## Saves Interpolated Maps and Exploratory Table to Geodatabase and POSTGIS (PostGres Database)

In [13]:
#converts IDW interpolated map to point
arcpy.conversion.RasterToPoint(
    in_raster="IDW_Temp",
    out_point_features= (arcpy.env.workspace + "\\IDW_Temp_Point"),
    raster_field="Value"
)
#saves IDW interpolated points to database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="IDW_Temp_Point",
    Output_Geodatabase= (arcpy.env.workspace + "\\Lab3.sde")
)
#converts Kriging interpolated map to point
arcpy.conversion.RasterToPoint(
    in_raster="Kriging_Temp",
    out_point_features= (arcpy.env.workspace + "\\Kriging_Temp_Point"),
    raster_field="Value"
)
#saves Kriging interpolated points to database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Kriging_Temp_Point",
    Output_Geodatabase= (arcpy.env.workspace + "\\Lab3.sde")
)
#converts Kernel interpolated map to point
arcpy.conversion.RasterToPoint(
    in_raster="Kernel_Temp",
    out_point_features= (arcpy.env.workspace + "\\Kernel_Temp_Point"),
    raster_field="Value"
)
#saves Kernel interpolated points to database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Kernel_Temp_Point",
    Output_Geodatabase= (arcpy.env.workspace + "\\Lab3.sde")
)
#saves exploratory table
arcpy.conversion.TableToGeodatabase(
    Input_Table="\\Lab3.gdb\\exploratoryinterpolation2",
    Output_Geodatabase= (arcpy.env.workspace + "\\Lab3.sde")
)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\ArcIILab2\\Lab3.sde'>

## Accuracy Assessment
#### Perform Accuracy Assessment on the testing 10% of data compared to the most accurate Interpolation.

In [15]:
#stores difference in ground truth (comparing interpolation estimate to actual point data) 
#kriging was best-- as exploration interpolation choose (highest accuracy in leave one out cross validation)
#defining paths
input_point_dataset = arcpy.env.workspace + "\\RandomSampledFeatures2" 
input_raster = arcpy.env.workspace + "\\Kriging_Temp"  
output_point_dataset = arcpy.env.workspace + "\\Temp_Accuracy"  
#step 1: Copy the point dataset (10%)
arcpy.CopyFeatures_management(input_point_dataset, output_point_dataset)
#step 2: Extract raster values to the copied points dataset
#field is value in our interpolation outputs and taking that field and extracting those values into a point dataset
#interpolation raster set + 10% test sample together into one dataset --two columns to directly subtract
extracted_field_name = "VALUE"  
arcpy.sa.ExtractValuesToPoints(
    in_point_features="RandomSampledFeatures2",
    in_raster="Kriging_Temp",
    out_point_features= arcpy.env.workspace + "\\Temp_Accuracy",
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY"
)
#names of columns
    ##temp: (testing data)
    ##RASTERVALU (interpolation data we pulled)
    ##Diff_Value (new field) (elevation minus RASTERVALU)
    ##this is prep to calculate for Diff_Value
point_value_field = "Temp"  
difference_field = "Diff_Value"  
extracted_field_name = "RASTERVALU"
#step 3: add a new field for the difference (aka calculate)
arcpy.management.CalculateField(
    in_table="Temp_Accuracy",
    field="Diff_Value",
    expression="!Temp!-!RASTERVALU!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
print("Done.")

Done.


## Saves Accuracy Assessment to Database

In [16]:
#saves to database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Temp_Accuracy",
    Output_Geodatabase= (arcpy.env.workspace + "\\Lab3.sde")
)

<Result 'C:\\Users\\Track\\OneDrive\\Documents\\ArcGIS\\Projects\\ArcIILab2\\Lab3.sde'>